In [ ]:
#!jupyter nbconvert --to script 1-preprocessing.ipynb

In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
drop_initially = ['pais_residencia', 'fecha_alta', 'ult_fec_cli_1t', 'conyuemp',
                  'canal_entrada', 'indfall', 'tipodom', 'nomprov']

In [3]:
import warnings

# Disable typecast warnings
warnings.filterwarnings('ignore')

na_values=['NA', 'nan']

dtype = {'tiprel_1mes':np.str, 'indrel_1mes':np.str, 'fecha_dato':np.str, 'ult_fec_cli_1t':np.str}

iter_csv = pd.read_csv('../data/raw/train.csv.zip', na_values = na_values, skipinitialspace = True, 
                       iterator=True, chunksize=100000, dtype=dtype)

df = pd.concat([chunk for chunk in iter_csv], ignore_index=True)
df.drop(drop_initially, axis=1, inplace=True)

In [7]:
test = df[df.renta.notnull()].head(10)
test

,fecha_dato,ncodpers,ind_empleado,sexo,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,H,35.0,0.0,6.0,1.0,1.0,A,S,N,29.0,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,V,23.0,0.0,35.0,1.0,1,I,S,S,13.0,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,V,23.0,0.0,35.0,1.0,1,I,S,N,13.0,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,H,22.0,0.0,35.0,1.0,1,I,S,N,50.0,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
5,2015-01-28,1050615,N,H,23.0,0.0,35.0,1.0,1,I,S,N,45.0,0.0,22220.04,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
6,2015-01-28,1050616,N,H,23.0,0.0,35.0,1.0,1,I,S,N,24.0,0.0,295590.36,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
7,2015-01-28,1050617,N,H,23.0,0.0,35.0,1.0,1,A,S,N,50.0,1.0,113316.66,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9,2015-01-28,1050620,N,H,23.0,0.0,35.0,1.0,1,I,S,N,10.0,0.0,113194.98,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
10,2015-01-28,1050621,N,V,23.0,0.0,35.0,1.0,1,I,S,N,50.0,0.0,72575.88,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
12,2015-01-28,1050623,N,H,23.0,0.0,35.0,1.0,1,A,S,N,49.0,1.0,113538.81,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [18]:
np.percentile(test.renta, [33, 66])

[22220.040000000001, 35548.739999999998, 72575.880000000005, 87218.100000000006, 113194.98, 113316.66, 113538.81, 119775.53999999999, 122179.11000000002, 295590.35999999999]
109515.822
113255.82


array([  86778.8334,  113525.481 ])

In [ ]:
[  87218.1    35548.74  122179.11  119775.54   22220.04  295590.36
  113316.66  113194.98   72575.88  113538.81]

In [ ]:
na_values=['NA', 'nan']
dtype = {'tiprel_1mes':np.str, 'indrel_1mes':np.str, 'fecha_dato':np.str}

iter_csv = pd.read_csv('../data/raw/test.csv.zip', na_values = na_values, skipinitialspace = True, 
                       iterator=True, chunksize=100000, dtype=dtype)

test = pd.concat([chunk for chunk in iter_csv], ignore_index=True)
test.drop(drop_initially, axis=1, inplace=True)

In [ ]:
test.head()

In [ ]:
products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [ ]:
segm_rent = defaultdict(set)
df.groupby(['segmento']).apply(lambda x: segm_rent[x.segmento.values[0]].add(x.renta.median()))

keys = []
vals = []

for k, v in segm_rent.items():
    keys.append(k)
    vals.append(list(v)[0])
    
fig, ax = plt.subplots()
ind = np.arange(len(keys))  # the x locations for the groups

ax.bar(ind, vals, align='center')
ax.set_xticks(ind)
ax.set_xticklabels(keys)
plt.show()

In [ ]:
prov_rent = defaultdict(set)
df.groupby(['cod_prov']).apply(lambda x: prov_rent[x.cod_prov.values[0]].add(x.renta.median()))

keys = []
vals = []

for k, v in prov_rent.items():
    keys.append(k)
    vals.append(list(v)[0])
    
fig, ax = plt.subplots(figsize=(20, 15))
ind = np.arange(len(keys))  # the x locations for the groups

ax.bar(ind, vals, align='center')
ax.set_xticks(ind)
ax.set_xticklabels(keys, rotation='vertical')
plt.show()

In [32]:
grouped = df.groupby(["segmento"]).agg({"renta":lambda x: np.median(x)})

In [33]:
grouped.head()

,renta
segmento,
01 - TOP,NaN
02 - PARTICULARES,NaN
03 - UNIVERSITARIO,NaN


In [ ]:
for (prov, segm) in grouped.index.get_values():
    df.ix[(df.renta.isnull()) & (df.cod_prov == prov) & (df.segmento == segm), 'renta'] = grouped.loc[prov, segm].values[0]
    df.ix[(df.renta.isnull()) & (df.cod_prov == prov), 'renta'] = grouped.renta.loc[prov, :].mean()
    df.ix[(df.renta.isnull()) & (df.segmento == segm), 'renta'] = grouped.renta.loc[grouped.index.get_level_values('segmento') == segm].mean()
    
    
    test.ix[(test.renta.isnull()) & (test.cod_prov == prov) & (test.segmento == segm), 'renta'] = grouped.loc[prov, segm].values[0]
    test.ix[(test.renta.isnull()) & (test.cod_prov == prov), 'renta'] = grouped.renta.loc[prov, :].mean()
    test.ix[(test.renta.isnull()) & (test.segmento == segm), 'renta'] = grouped.renta.loc[grouped.index.get_level_values('segmento') == segm].mean()
    
df.loc[df.renta.isnull(), 'renta'] = df.renta.median()
test.loc[test.renta.isnull(), 'renta'] = df.renta.median()

In [ ]:
df = df[df.age.notnull()]
df = df[df.sexo.notnull()]
df = df[df.fecha_dato.isin(['2015-05-28', '2015-06-28', '2016-05-28', '2016-06-28'])]
df.sort_values(['ncodpers', 'fecha_dato'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
def sub_code_and_toint(text):
    return 0.0 if pd.isnull(text) else float(str(text)[1:2])
    
def preprocess_features(X):        
    dummies = ['ind_empleado', 'indrel_1mes', 'indrel_1mes', 'tiprel_1mes', 'segmento']
    X.drop(columns_to_drop, axis=1, inplace=True) 

    # sexo
    X.sexo.replace(to_replace=['V', 'H'], value=[1.0, 0.0], inplace=True)
    
    # ind_empleado
    ind_empleado = pd.get_dummies(X.ind_empleado, prefix = 'ind_empleado')
    X = pd.concat([X, ind_empleado], axis=1)
    
    # ind_nuevo
    X.ix[(X.ind_nuevo.isnull()) & (X.antiguedad.isnull()), 'ind_nuevo'] = 0
    X.ix[(X.ind_nuevo.isnull()) & (X.antiguedad < 6), 'ind_nuevo'] = 1
    X.ix[(X.ind_nuevo.isnull()) & (X.antiguedad >= 6), 'ind_nuevo'] = 0    
    
    # indrel
    X.indrel.replace(to_replace=[99.], value=[0.], inplace=True)
    
    # indrel_1mes
    X.indrel_1mes.replace(to_replace=['P', float('nan')], value=[5., 6.], inplace=True)
    X.indrel_1mes = X.indrel_1mes.astype(np.float16)
    indrel_1mes = pd.get_dummies(X.indrel_1mes, prefix = 'indrel_1mes')
    X = pd.concat([X, indrel_1mes], axis=1)   
    
    # tiprel_1mes
    X.tiprel_1mes.replace(to_replace=[float('nan')], value=['N'], inplace=True)
    tiprel_1mes = pd.get_dummies(X.tiprel_1mes, prefix = 'tiprel_1mes')
    X = pd.concat([X, tiprel_1mes], axis=1)   
    
    # indresi
    X.indresi.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True) 
    
    # indext
    X.indext.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True) 
    
    # cod_prov
    X.cod_prov.replace(to_replace=[float('nan')], value=[0.], inplace=True)

    # segmento
    segmento = pd.get_dummies(X.segmento, prefix = 'segmento')
    X = pd.concat([X, segmento], axis=1)  
    
    X.drop(dummies, axis=1, inplace=True)   
        
    return X

In [ ]:
df = preprocess_features(df)
df.ix[df[products].isnull().any(axis=1)] = 0

test = preprocess_features(test)
test.loc[test.sexo.isnull(), 'sexo'] = 0

In [ ]:
df.to_csv('../data/processed/train-processed.csv.gz', index=False, compression='gzip')

In [ ]:
test.to_csv('../data/processed/test-processed.csv.gz', index=False, compression='gzip')